In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/interpolated-data-2000/Interpolated_2000.csv
/kaggle/input/extra-data-concat-df/concatenated_sentiment_ff.csv
/kaggle/input/using-financial-news-to-predict-the-s-p-500-cs-v2/sentiment_gpt4_only.csv
/kaggle/input/using-financial-news-to-predict-the-s-p-500-cs-v2/train.csv
/kaggle/input/using-financial-news-to-predict-the-s-p-500-cs-v2/Metadata.csv
/kaggle/input/using-financial-news-to-predict-the-s-p-500-cs-v2/test.csv


# 1 Preprocessing the Extra Dataset

Other packages

In [44]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier as RFS
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [45]:
#sentiment is the dataframe that has the extra sentiment collected
#extra_df is the dataframe with the extra yfinance data (already interpolated earlier)
train=pd.read_csv('/kaggle/input/using-financial-news-to-predict-the-s-p-500-cs-v2/train.csv')
sentiment=pd.read_csv('/kaggle/input/using-financial-news-to-predict-the-s-p-500-cs-v2/sentiment_gpt4_only.csv')
sentiment = pd.read_csv('/kaggle/input/extra-data-concat-df/concatenated_sentiment_ff.csv')
extra_df = pd.read_csv('/kaggle/input/interpolated-data-2000/Interpolated_2000.csv')

In [46]:
def signal_func(L):
    S=0
    for ch in L:
        if ch=='positive':
            S+=1
        if ch=='negative':
            S-=1
    return S/len(L)

In [47]:
sentiment.sort_values(by = 'Date' ,inplace = True)
sentiment.drop_duplicates(inplace = True)
sentiment_result = sentiment.groupby('Date')['Sentiment'].apply(signal_func).reset_index()
sentiment_result.head()

,Date,Sentiment
0,2000-01-04,1.0
1,2000-01-13,1.0
2,2000-01-24,-1.0
3,2000-01-27,-1.0
4,2000-02-02,1.0


In [48]:
test_cols_req = list(extra_df.columns)
test_cols_req.remove('target')
tickers = test_cols_req
tickers.remove('Date')
tickers.remove('SI=F')

In [49]:
scaler = MinMaxScaler(feature_range = (0,1))

In [50]:
extra_df[tickers] = scaler.fit_transform(extra_df[tickers])
extra_df.drop(['SI=F','HG=F','^DJI','^DJI','^IXIC','^RUT','^TNX','^TYX','^VIX'],axis=1,inplace = True)
concat_train_df = pd.merge(sentiment_result, extra_df, on='Date', how='inner')

In [51]:
concat_train_df.head()

,Date,Sentiment,^GSPC,CL=F,GC=F,JPY=X,target
0,2000-10-03,0.0,0.843926,0.114362,0.011716,0.550192,1
1,2000-10-12,-1.0,0.735129,0.145885,0.015125,0.530455,1
2,2000-10-17,-1.0,0.757849,0.121245,0.011361,0.533225,0
3,2000-11-03,-1.0,0.844185,0.118507,0.006746,0.516604,1
4,2000-11-07,1.0,0.850015,0.124765,0.006533,0.518508,0


## Training NN

In [52]:
import tensorflow as tf

##### Verifying that the dataframe doesnt have any missing values

In [53]:
concat_train_df.drop(['Date','target'],axis=1).isna().any()

Sentiment    False
^GSPC        False
CL=F         False
GC=F         False
JPY=X        False
dtype: bool

### Regressive Neural Network

In [54]:
x_train_1 = concat_train_df.drop(['Date','target'],axis=1).to_numpy()
y_train_1 = concat_train_df[['target']].to_numpy()

Reshaping the dataset, since LSTM layers require 2 dimensional input

In [55]:
x_train_1 = x_train_1.reshape(x_train_1.shape[0],1,x_train_1.shape[1])

In [56]:
X_train, X_test, y_train, y_test = train_test_split(x_train_1,y_train_1, test_size=0.2, random_state=42)

In [57]:
x_ip_1_tensor = tf.convert_to_tensor(X_train)
y_op_1_tensor = tf.convert_to_tensor(y_train)

In [58]:
x_ip_1_tensor.shape,y_op_1_tensor.shape 

(TensorShape([704, 1, 5]), TensorShape([704, 1]))

## First Neural Network Structure

4 layers of LSTM, with 50 units each along with a dense layer of 1 neuron at the end

In [76]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

model = Sequential(name='regressive_model_1')
model.add(LSTM(units=50,return_sequences=True,input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "regressive_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_24 (LSTM)                  │ (None, 1, 50)          │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 1, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_25 (LSTM)                  │ (None, 1, 50)          │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 1, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_26 (LSTM)                  │ (None, 1, 50)          │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 1, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_27 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 71,851 (280.67 KB)

 Trainable params: 71,851 (280.67 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x_ip_1_tensor,y_op_1_tensor,epochs=10,batch_size=32);

## F1 Score

In [70]:
pred_y = model.predict(X_test)
pred_y = np.round(pred_y)
f1_score(pred_y,y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step


0.5123152709359605

## Second Neural Network Structure

Much more complicated, as it involves 2 LSTM layers going forward then backwards, and 4 Dense layers going with decreasing set of units

In [71]:
learning_rate = 0.0001
loss_fn = tf.keras.losses.MeanSquaredError()
initial_state = tf.keras.initializers.Constant(value=0.1)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
metrics = ['accuracy']
seq_len = x_ip_1_tensor.shape[1]
no_cols = x_ip_1_tensor.shape[2]

input_fin_data = tf.keras.layers.Input(shape=(seq_len,no_cols))

lstm_1 = tf.keras.layers.LSTM(120,activation = 'relu', return_sequences=True, dropout = 0.2)
output_1 = lstm_1(input_fin_data)

lstm_2 = tf.keras.layers.LSTM(120,activation = 'relu',go_backwards = True, return_sequences = True)
output_2 = lstm_2(output_1)

gru_1 = tf.keras.layers.GRU(80,activation = 'relu',dropout = 0.2, return_sequences = True)
output_3 = gru_1(output_2)

gru_2 = tf.keras.layers.GRU(80,activation = 'relu',dropout = 0.2, go_backwards = True)
output_4 = gru_2(output_3)

#net_joined = tf.keras.layers.concatenate([output_2, input_sentiment], axis=-1)

dense_layer_1 = tf.keras.layers.Dense(40, activation = 'relu')
output_7 = dense_layer_1(output_4)

dense_layer_pre = tf.keras.layers.Dense(30, activation = 'relu')
output_8 = dense_layer_pre(output_7)

dense_layer_2 = tf.keras.layers.Dense(20, activation = 'relu')
output_9 = dense_layer_2(output_8)

dense_layer = tf.keras.layers.Dense(1, activation='sigmoid')
net_output = dense_layer(output_9)

neural_nw_2 = tf.keras.Model(inputs=[input_fin_data], outputs=net_output, name = 'regressive_model_2')
neural_nw_2.compile(loss = loss_fn, optimizer = optimizer, metrics = metrics)

neural_nw_2.summary()

Model: "regressive_model_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1, 5)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 1, 120)         │        60,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 1, 120)         │       115,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 1, 80)          │        48,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 80)             │        38,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 40)             │         3,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 30)             │         1,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 20)             │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 268,631 (1.02 MB)

 Trainable params: 268,631 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

### Training by Automatic Gradient Descent

with 100 iterations of batch sizes 200 each

In [ ]:
batch_size = 200
num_iters = 100

optimizer = neural_nw_2.optimizer
loss_fn = neural_nw_2.loss

for iteration in range(num_iters+1):

    indices = np.arange(x_ip_1_tensor.shape[0])
    np.random.shuffle(indices)

    shuffled_X_1 = tf.gather(x_ip_1_tensor, indices[:batch_size+1])
    shuffled_Y = tf.gather(y_op_1_tensor, indices[:batch_size+1])

    with tf.GradientTape() as tape:
        predictions = neural_nw_2(shuffled_X_1,training=True)
        loss = loss_fn(shuffled_Y, predictions)

    gradients = tape.gradient(loss, neural_nw_2.trainable_variables)

    optimizer.apply_gradients(zip(gradients, neural_nw_2.trainable_variables))

    if iteration % 100 == 0 and iteration!=0:
        print(f'Batch {iteration}/{num_iters}, Loss: {loss:.4f}')

## F1 Score

In [72]:
pred_y = neural_nw_2.predict([X_test])
pred_y = np.round(pred_y);
f1_score(pred_y,y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step


0.0

## Predictions

In [ ]:
test_df = pd.read_csv('/kaggle/input/using-financial-news-to-predict-the-s-p-500-cs-v2/test.csv')

In [ ]:
test_df=pd.merge(sentiment_result, test_df, on='Date', how='inner')
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df.set_index('Date',inplace=True)
test_df = test_df.interpolate(method='time')
test_df.drop(columns=['EURUSD=X','XWD.TO','SI=F','HG=F',
                      '^DJI','^DJI','^IXIC','^RUT','^TNX','^TYX','^VIX'],inplace=True)

In [ ]:
test_df.head()

#### Reordering the columns to match the train dataset

In [ ]:
test_cols_req = list(concat_train_df.columns)
test_cols_req.remove('target')
test_cols_req.remove('Date')

In [ ]:
order_df = pd.DataFrame({
    'order': test_cols_req 
})
desired_order = order_df['order'].tolist()
test_df = test_df[desired_order]

##### Final Predictions

In [ ]:
x_test_final = test_df.to_numpy()
x_test_final = x_test_final.reshape(x_test_final.shape[0],1,x_test_final.shape[1])
x_test_final.shape

In [63]:
predictions = np.round(neural_nw_2.predict([x_test_final])).astype(int)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [67]:
submission_df = pd.DataFrame(predictions,columns=['target'])
submission_df = submission_df.reset_index().rename(columns={'index': 'id'})
submission_df.head()

,id,target
0,0,1
1,1,1
2,2,1
3,3,1
4,4,0


In [69]:
submission_df.to_csv('submission.csv',index = False)